In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import  ToTensor, Normalize



transform = transforms.Compose([
    ToTensor(),
    Normalize((0.1307, ), (0.3081,))
])

In [65]:
train_dataset = torchvision.datasets.MNIST(
    root='./data/train',
    download=True,
    train=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data/test',
    download=True,
    train=False,
    transform=transform
)

In [66]:
BATCH_SIZE = 512
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    shuffle=True,
    num_workers=2,
    batch_size=BATCH_SIZE
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    shuffle=False,
    num_workers=2,
    batch_size=BATCH_SIZE
)

In [67]:
import torch.nn as nn
import torch.optim as optim
from cnn.Net import Net

input_dim = (28,28)
net = Net(
    input_dim=input_dim,
    num_classes=10
)

lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lr=lr, params=net.parameters())

In [68]:
from tqdm import  tqdm
EPOCHS = 32
epochs_tqdm = tqdm(range(EPOCHS), desc="Training", unit=f"batch-{BATCH_SIZE}")
for epoch in epochs_tqdm:
    total_loss = 0
    total_items = 0
    net.train()
    for images, labels in train_loader:
        optimizer.zero_grad()

        out = net(images)
        loss = criterion(out, labels)

        loss.backward()
        optimizer.step()

        total_loss += torch.sum(loss).item()
        total_items += images.shape[0]

    epochs_tqdm.set_postfix(train_loss=total_loss/total_items)

Training: 100%|██████████| 32/32 [05:22<00:00, 10.09s/batch-512, train_loss=2.38e-5]


In [69]:
torch.tensor([123]).shape

torch.Size([1])

In [70]:
num_correct = 0
num_item = 0
net.eval()
for images, labels in test_loader:
    out = net(images)
    _,predicts = out.max(dim=1)
    num_correct += (predicts == labels).sum().item()
    num_item+=images.shape[0]
num_correct/num_item


0.993

In [71]:
num_correct = 0
num_item = 0
net.eval()
for images, labels in train_loader:
    out = net(images)
    _,predicts = out.max(dim=1)
    num_correct += (predicts == labels).sum().item()
    num_item+=images.shape[0]
num_correct/num_item

0.9996833333333334